# NUEVOS DATOS DE NIGERIA

link al dataset: https://zenodo.org/record/1252141

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [92]:
metadata = pd.read_csv('C:\\Users\\47575909\\Desktop\\metadata_nigeria.csv')
metadata

subject.id  recordedPeriod        startTime  session.id first_condition  \
0             6             270  26/9/2016 13:13           1            open   
1             9             271  26/9/2016 13:30           1          closed   
2            10             272  26/9/2016 13:36           1            open   
3            11             274  26/9/2016 13:42           2          closed   
4            11               1  26/9/2016 13:42           1          closed   
5            12             271  26/9/2016 13:47           1            open   
6            40             271  26/9/2016 16:56           1            open   
7            49             268  27/9/2016 14:29           1          closed   
8            63             268   30/9/2016 9:45           1            open   
9            64             271   30/9/2016 9:51           1          closed   
10           65              59  30/9/2016 10:00           2            open   
11           65             266   30/9/2016 9:56           1            open   
12           66             269  30/9/2016 10:02           1          closed   
13           67             268  30/9/2016 10:08           1            open   
14           68             268  30/9/2016 10:14           1          closed   
15           69             289  30/9/2016 10:20           1            open   
16           70             268  30/9/2016 10:26           1          closed   
17           71             268  30/9/2016 10:32           1            open   
18           72             268  30/9/2016 10:37           1          closed   
19           73             268  30/9/2016 10:42           1            open   
20           74             269  30/9/2016 10:48           1          closed   
21           75             266  30/9/2016 10:53           1            open   
22           76             269  30/9/2016 10:59           1          closed   
23           77             269  30/9/2016 11:05           1            open   
24           78             268  30/9/2016 11:10           1          closed   
25           79             268  30/9/2016 11:15           1            open   
26           80             268  30/9/2016 11:21           1          closed   
27           81             268  30/9/2016 11:26           1            open   
28           82             268  30/9/2016 11:32           1          closed   
29           83             268  30/9/2016 11:37           1            open   
30           84             251  30/9/2016 11:42           1          closed   
31           85             268  30/9/2016 11:48           1            open   
32           86             268  30/9/2016 11:53           1          closed   
33           87             268  30/9/2016 11:58           1            open   
34           88             268  30/9/2016 12:04           1          closed   
35           89             268  30/9/2016 12:09           1            open   
36           90             268  30/9/2016 12:15           1          closed   
37           91             264  30/9/2016 12:20           1            open   
38           92             268  30/9/2016 12:26           1          closed   
39           93             269  30/9/2016 12:32           1            open   
40           95             268  30/9/2016 12:44           1            open   
41           96             268  30/9/2016 12:50           1          closed   
42           97             268  30/9/2016 12:55           1            open   
43           98             249  30/9/2016 13:01           1          closed   
44           99             268  30/9/2016 13:06           1            open   
45          100             268  30/9/2016 13:12           1          closed   
46          101             268  30/9/2016 13:17           1            open   
47          102             268  30/9/2016 13:22           1          closed   
48          103             268  30/9/2016 13:27           1            open   
49   

In [93]:
#veo la mierda de los ids
groups = metadata.groupby('subject.id')

grp_keys = list(groups.groups.keys())
print(len(grp_keys))

222


In [94]:
metadata = metadata.drop([2, 4, 10, 11, 15, 64], axis=0)  # Elimina las filas que no me sirven
metadata.reset_index(drop=True, inplace=True)  # Restablece los índices y aplica los cambios

metadata
#SINO ELIMINA ESA PERSONA Y YA ESTA QCY

subject.id  recordedPeriod        startTime  session.id first_condition  \
0             6             270  26/9/2016 13:13           1            open   
1             9             271  26/9/2016 13:30           1          closed   
2            11             274  26/9/2016 13:42           2          closed   
3            12             271  26/9/2016 13:47           1            open   
4            40             271  26/9/2016 16:56           1            open   
5            49             268  27/9/2016 14:29           1          closed   
6            63             268   30/9/2016 9:45           1            open   
7            64             271   30/9/2016 9:51           1          closed   
8            66             269  30/9/2016 10:02           1          closed   
9            67             268  30/9/2016 10:08           1            open   
10           68             268  30/9/2016 10:14           1          closed   
11           70             268  30/9/2016 10:26           1          closed   
12           71             268  30/9/2016 10:32           1            open   
13           72             268  30/9/2016 10:37           1          closed   
14           73             268  30/9/2016 10:42           1            open   
15           74             269  30/9/2016 10:48           1          closed   
16           75             266  30/9/2016 10:53           1            open   
17           76             269  30/9/2016 10:59           1          closed   
18           77             269  30/9/2016 11:05           1            open   
19           78             268  30/9/2016 11:10           1          closed   
20           79             268  30/9/2016 11:15           1            open   
21           80             268  30/9/2016 11:21           1          closed   
22           81             268  30/9/2016 11:26           1            open   
23           82             268  30/9/2016 11:32           1          closed   
24           83             268  30/9/2016 11:37           1            open   
25           84             251  30/9/2016 11:42           1          closed   
26           85             268  30/9/2016 11:48           1            open   
27           86             268  30/9/2016 11:53           1          closed   
28           87             268  30/9/2016 11:58           1            open   
29           88             268  30/9/2016 12:04           1          closed   
30           89             268  30/9/2016 12:09           1            open   
31           90             268  30/9/2016 12:15           1          closed   
32           91             264  30/9/2016 12:20           1            open   
33           92             268  30/9/2016 12:26           1          closed   
34           93             269  30/9/2016 12:32           1            open   
35           95             268  30/9/2016 12:44           1            open   
36           96             268  30/9/2016 12:50           1          closed   
37           97             268  30/9/2016 12:55           1            open   
38           98             249  30/9/2016 13:01           1          closed   
39           99             268  30/9/2016 13:06           1            open   
40          100             268  30/9/2016 13:12           1          closed   
41          101             268  30/9/2016 13:17           1            open   
42          102             268  30/9/2016 13:22           1          closed   
43          103             268  30/9/2016 13:27           1            open   
44          104             268  30/9/2016 13:33           1          closed   
45          105             268  30/9/2016 13:38           1            open   
46          106             269  30/9/2016 13:43           1          closed   
47          107             268  30/9/2016 13:48           1            open   
48          108             268  30/9/2016 13:55           1          closed   
49   

In [95]:
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('first_condition', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('remarks', axis=1)
metadata = metadata.drop('session.id', axis=1)

metadata['csv.file'] = metadata['csv.file'].str.replace('.csv.gz', '.csv')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(metadata)

     subject.id     Group          csv.file
0             6   control    signal-6-1.csv
1             9   control    signal-9-1.csv
2            11   control   signal-11-2.csv
3            12   control   signal-12-1.csv
4            40   control   signal-40-1.csv
5            49   control   signal-49-1.csv
6            63   control   signal-63-1.csv
7            64   control   signal-64-1.csv
8            66   control   signal-66-1.csv
9            67   control   signal-67-1.csv
10           68   control   signal-68-1.csv
11           70   control   signal-70-1.csv
12           71   control   signal-71-1.csv
13           72   control   signal-72-1.csv
14           73   control   signal-73-1.csv
15           74   control   signal-74-1.csv
16           75   control   signal-75-1.csv
17           76   control   signal-76-1.csv
18           77   control   signal-77-1.csv
19           78   control   signal-78-1.csv
20           79   control   signal-79-1.csv
21           80   control   sign

In [8]:
# Archivos en la segunda lista (directorio)
archivos = os.listdir('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria')

# Encontrar el archivo faltante en la primera lista
archivo_faltante = [archivo for archivo in archivos if archivo not in metadata['csv.file']]

print(f"El archivo que falta es: {archivo_faltante[0]}")

El archivo que falta en la primera lista es: signal-10-1.csv


In [6]:
'''dataset = pd.DataFrame()

for archivo_csv in os.listdir('C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'):
    if archivo_csv.endswith('.csv'):
        archivo_csv = archivo_csv.replace(' ', '')
        
        # Obtener el nombre de archivo sin la extensión .csv
        archivo_base = archivo_csv.replace('.csv', '')

        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda
        matching_rows = metadata.loc[metadata['csv.file'] == archivo_base]

        if not matching_rows.empty:  # Verifica si se encontraron coincidencias
            subject_id, label = matching_rows[['subject.id', 'Group']].values[0]

            # Leer el archivo CSV de EEG
            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))

            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG
            eeg_data['subject.id'] = subject_id
            eeg_data['Group'] = label

            # Agregar los datos del EEG al conjunto de datos combinado
            dataset = pd.concat([dataset, eeg_data])

            print(f"Se encontró una coincidencia para {archivo_csv}: subject_id={subject_id}, Group={label}")
        else:
            print(f"No se encontraron coincidencias para {archivo_csv}")

# Puedes guardar el conjunto de datos combinado en un nuevo archivo CSV si lo deseas
dataset.to_csv('new_eeg_data.csv', index=False)'''

'dataset = pd.DataFrame()\n\nfor archivo_csv in os.listdir(\'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\'):\n    if archivo_csv.endswith(\'.csv\'):\n        archivo_csv = archivo_csv.replace(\' \', \'\')\n        \n        # Obtener el nombre de archivo sin la extensión .csv\n        archivo_base = archivo_csv.replace(\'.csv\', \'\')\n\n        # Verifica si hay filas en metadata que cumplen con la condición de búsqueda\n        matching_rows = metadata.loc[metadata[\'csv.file\'] == archivo_base]\n\n        if not matching_rows.empty:  # Verifica si se encontraron coincidencias\n            subject_id, label = matching_rows[[\'subject.id\', \'Group\']].values[0]\n\n            # Leer el archivo CSV de EEG\n            eeg_data = pd.read_csv(os.path.join(directorio_eeg, archivo_csv))\n\n            # Agregar las columnas de subject ID y etiqueta al DataFrame de EEG\n            eeg_data[\'subject.id\'] = subject_id\n            eeg_data[\'Group\'] = label\n\n            # Agregar los da

In [96]:
path = 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria'
all_files = glob.glob(os.path.join(path, "*.csv"))

def alfanumerico_key(item):
    partes = re.split(r'(\d+)', item)
    partes[1::2] = map(int, partes[1::2])
    return partes

all_files = sorted(all_files, key=alfanumerico_key)

def add_id_y(all_files, metadata):
    modified_dataframes_list = []

    for filename, subject_id, group in zip(all_files, metadata['subject.id'], metadata['Group']):
        df = pd.read_csv(filename, index_col=None, header=0)
        dataframe = df.iloc[:, 1:15].copy()
        # Obtén el número del archivo a partir del nombre
        file_number = int(re.search(r'(\d+)', os.path.basename(filename)).group(1))
        dataframe['subject_id'] = subject_id
        dataframe['y'] = group
        modified_dataframes_list.append(dataframe)

    return modified_dataframes_list

In [97]:
all_files

['C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-6-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-9-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-11-2.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-12-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-40-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-49-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-63-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-64-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-66-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-67-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-68-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-70-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-71-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-72-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Nigeria\\signal-73-1.csv',
 'C:\\Users\\47575909\\Desktop\\EEGs_Niger

In [98]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
    dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_new_data.csv', index=False)

In [99]:
len(all_dataframes)

7288960

In [100]:
all_dataframes.loc[7288959]

AF3                 4154
AF4          -260.652632
F3                  4206
F4           -331.421053
F7                 181.4
F8                  3834
FC5          -289.989474
FC6                 4025
O1                  4250
O2                 169.6
P7            -76.042105
P8                  4237
T7                  3847
T8            -90.915789
subject_id           638
y               epilepsy
Name: 7288959, dtype: object

In [20]:
id_list = sorted(all_dataframes['subject_id'].unique())

print(id_list)

[6, 9, 10, 11, 12, 40, 49, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625,

In [ ]:
#SPLIT DATOS PARA PREDICT

In [2]:
data = np.load(f'C:\\Users\\47575909\\Desktop\\X_predict_epilepsy.npy')

In [3]:
midpoint = len(data) // 2
half = data[:midpoint]
np.save('X_predict_epilepsy_half.npy', half)